<a href="https://colab.research.google.com/github/VictorFu0717/AI/blob/main/w2v1215.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/mtkdl/raw/master/pttexsmall.csv"
urlretrieve(url, "ptt.csv")

In [ ]:
import jieba

url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.big")
url = "https://github.com/Elwing-Chou/mtkdl/raw/master/ptt_dic.txt"
urlretrieve(url, "dict.ptt")

jieba.set_dictionary("dict.big")
jieba.load_userdict("dict.ptt")

In [ ]:
import pandas as pd
df = pd.read_csv("ptt.csv", encoding="utf-8")
df

In [ ]:
#去除https跟標點符號
import re
def process(content):
    content = re.sub(r'https?:\/\/.*[\r\n]*', '', content)
    punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…~/ －＊➜■─★☆=@<>◉é''')
    cut = filter(lambda x: x not in punct, jieba.cut(content))
    return " ".join(cut)
df["content"] = df["content"].apply(process)
df

In [ ]:
with open("lines.txt", "w", encoding="utf-8") as f:
    for content in df["content"]:
        f.write(content)
        f.write("\n")

In [ ]:
from gensim.models.word2vec import LineSentence
from gensim.models.word2vec import Word2Vec
sentences = LineSentence("lines.txt")
model = Word2Vec(sentences)

In [ ]:
# model.wv.vocab
model.wv["肥宅"]
# model.wv["Elwing"]

In [ ]:
#找出相似詞
model.wv.most_similar("肥宅")

In [ ]:
#用tfidf再加權平均
import numpy as np
import jieba.analyse
encodings = []
for content in df["content"]:
    tags = jieba.analyse.extract_tags(content, 
                      topK=None, 
                      withWeight=True)
    total = np.zeros((100, ))
    weight = 0.0
    for word, tfidf in tags:
        if word in model.wv.vocab:
            total = total + tfidf * model.wv[word]
            weight = weight + tfidf
    encodings.append(total / weight)
print(len(encodings))

In [ ]:
with open("ptt.wv", "w", encoding="utf-8") as f:
    f.write("{} {}\n".format(20000, 100))
    for i, wv in enumerate(encodings):
        wv = " ".join(map(str, wv))
        f.write("{} {}\n".format(i, wv))

In [ ]:
from gensim.models import KeyedVectors
d2v = KeyedVectors.load_word2vec_format("ptt.wv")

In [ ]:
tok = input("Enter(0-19999):")
data = df.iloc[int(tok)]
print("標題:", data["title"])
print("內容:", data["content"])
print("-" * 30)
tok2, sim = d2v.most_similar(tok)[0]
print("相似度:", sim)
data = df.iloc[int(tok2)]
print("標題:", data["title"])
print("內容:", data["content"])